In [1]:
import pandas as pd
from scipy.stats import norm
from numpy import sqrt

table_a = 'TableA'
table_b = 'TableB'
candidate_set = 'CandidateSet'
prediction_set = 'PredictionList'

dfa = pd.read_csv(table_a)
dfb = pd.read_csv(table_b)
dfc = pd.read_csv(candidate_set)
dfp = pd.read_csv(prediction_set)

shuffled_dfc = dfc.sample(frac=1)
# shuffled_dfc.to_csv('shuffled_dfc', index=False)

In [2]:
import re
at = 'American War: A Novel'
bt = 'American War'
atitle = re.sub('[^0-9a-zA-Z]+', ' ', at).split(' ')
btitle = re.sub('[^0-9a-zA-Z]+', ' ', bt).split(' ')
common_words = list(set(atitle) & set(btitle))
total_words = list(set(atitle) | set(btitle))
fraction = len(common_words) / len(total_words)
fraction

0.5

In [3]:
import re
def blocking_rules():
    new_cs = {}
    j = 0
    for i in range(len(shuffled_dfc)):
        tuple = shuffled_dfc.iloc[i, :]
        idxa = tuple.A_id; idxb = tuple.B_id
        rowa = dfa.iloc[idxa, :]
        rowb = dfb.iloc[idxb, :]
        # Blocking Rule 1: Block all with missing `title` value
        if type(rowa['title']) == str and type(rowb['title']) == str and len(rowa['title']) > 0 and len(rowb['title']) > 0:
            # Blocking Rule 2: Block tuple pairs with non-matching `book_format`
            if rowa['book_format'] == rowb['book_format']:
                # Blocking Rule 3: Fraction of common words in title of tuple pairs > threshold(0.25)
                atitle = re.sub('[^0-9a-zA-Z]+', ' ', rowa['title']).split(' ')
                btitle = re.sub('[^0-9a-zA-Z]+', ' ', rowb['title']).split(' ')
                common_words = list(set(atitle) & set(btitle))
                total_words = list(set(atitle) | set(btitle))
                fraction = len(common_words) / len(total_words)
                if fraction > 0.10:
                    new_cs[j] = tuple
                    j = j+1
        
    updated_cdf = pd.DataFrame.from_dict(new_cs)
    updated_cdf = updated_cdf.T
    updated_cdf.to_csv('updated_cdf_afterblocking3.csv', index=False)
    
    return len(new_cs)
blocking_rules()

1520

In [4]:
shuffled_dfc = pd.read_csv('updated_cdf_afterblocking3.csv')
print('Len of new shuffled_dfc', len(shuffled_dfc))

Len of new shuffled_dfc 1520


In [5]:

def get_sample(start=0, n=50):
    matching_dict = {}
    current_sample = {}
    k = start
    cols = dfa.columns.tolist()
    cols.remove('Up_System')
    cols.remove('_id')
    cols = ['_id', 'Up_System', 'predicted', 'manual'] + cols
    for i in range(len(shuffled_dfc[start:start+n])):
        tuple = shuffled_dfc.iloc[start + i, :]
        idxa = tuple.A_id; idxb = tuple.B_id
        rowa = dfa.iloc[idxa, :]
        rowb = dfb.iloc[idxb, :]
        rowa['Up_System'] = str(start + i) + 'A'
        rowb['Up_System'] = str(start + i) + 'B'
        rowa['manual'] = ''
        rowb['manual'] = ''
        matching = dfp[(dfp['id1'] == rowa['_id']) & (dfp['id2'] == rowb['_id'])]
        if len(matching) == 1 and matching['id1'].values[0] == rowa['_id'] and matching['id2'].values[0] == rowb['_id']:
            matching_dict[str(rowa['_id']) + '_' + str(rowb['_id'])] = True
            rowa['predicted'] = True
            rowb['predicted'] = True
        else:
            rowa['predicted'] = False
            rowb['predicted'] = False
        #print(pd.Series.to_dict(rowa))
        current_sample[k] = pd.Series.to_dict(rowa)
        current_sample[k+1] = pd.Series.to_dict(rowb)
        k += 2

    Sdf = pd.DataFrame.from_dict(current_sample)
    Sdf = Sdf.T
    #Sdf.to_csv('run'+str(start) + '_' + str(start+n) + '.csv', index=False, columns=cols)
    Sdf.to_csv('all1520.csv', index=False, columns=cols)
    return len(matching_dict)
get_sample(0, 2000)

0


/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

1
2
3


/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
27

{'385_2518': True,
 '2047_2658': True,
 '3408_1530': True,
 '3880_1327': True,
 '2874_2528': True,
 '1912_4867': True,
 '2173_4518': True,
 '2005_3720': True,
 '4037_311': True,
 '352_598': True,
 '499_3510': True,
 '1859_4005': True,
 '2093_794': True,
 '307_2372': True,
 '2070_4470': True,
 '2866_3622': True,
 '3684_4871': True,
 '2265_1591': True,
 '3395_3208': True,
 '1519_1279': True,
 '1741_2238': True,
 '3349_2456': True,
 '1417_3962': True,
 '1738_4814': True,
 '2008_4771': True,
 '3626_299': True,
 '251_970': True,
 '2207_4903': True,
 '2630_2523': True,
 '2040_2038': True,
 '935_4092': True,
 '3686_4312': True,
 '1683_5047': True,
 '2447_4936': True,
 '1233_4188': True,
 '3944_701': True,
 '2281_5081': True,
 '4120_3202': True,
 '3598_1355': True,
 '757_885': True,
 '1701_5344': True,
 '3562_3634': True,
 '3523_4323': True,
 '1928_2200': True,
 '428_3072': True,
 '3825_3508': True,
 '1815_4012': True,
 '2365_4164': True,
 '125_2173': True,
 '2161_1379': True,
 '2373_1176': Tr

In [6]:
ldf = pd.read_csv('labelled400.csv')
labelled_pairs = {}
counter = 0
for i in range(0, len(ldf), 2):
    t1 = ldf.iloc[i, :]['_id']
    t2 = ldf.iloc[i+1, :]['_id']
    label = ldf.iloc[i, :]['manual']
    if label == 1:
        label = True
    else:
        label = False
    labelled_pairs[counter] = {'id1': t1, 'id2': t2, 'label': label}
    counter += 1

lpdf = pd.DataFrame.from_dict(labelled_pairs)
lpdf = lpdf.T
lpdf.to_csv('labelled_pairs.csv', index=False)